In [12]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
# import squidpy as sq

from scipy.spatial import cKDTree
# import muon as mu
import mudata as mu

import spatialdata as sd

In [2]:
## load slide
slide = "B1"
exp = "V11L12-109" # V11T17-102 V11L12-109 V11T16-085
slide_path = f"data/VicariEtAl/sma/{exp}/{exp}_{slide}/output_data"

In [3]:
mdata = mu.read_h5mu(os.path.join(slide_path, "sma.h5mu"))
rna = mdata.mod["rna"]
msi = mdata.mod["msi"]

/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1900: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


In [4]:
rna_pos = rna.obsm['spatial'].copy()
msi_pos = msi.obsm['spatial'].copy()

try spatialdata

In [5]:
from spatialdata.models import TableModel, ShapesModel
from anndata import AnnData
import numpy as np
from spatialdata import SpatialData

In [6]:
# shapes 
rna_locations = ShapesModel.parse(rna_pos, geometry=0, radius=1)
msi_locations = ShapesModel.parse(msi_pos, geometry=0, radius=1)

mdata.obs['region'] = (['rna'] * rna_pos.shape[0] + ['msi'] * msi_pos.shape[0]).copy()
mdata.obs['instance_id'] = mdata.obs.index

mdata = TableModel.parse(mdata, region=['msi', 'rna'], region_key='region', instance_key='instance_id')
sdata = SpatialData(shapes={'msi':msi_locations, 'rna':rna_locations}, table=mdata)

/home/dbdimitrov/anaconda3/envs/spatialdata/lib/python3.10/site-packages/spatialdata/models/models.py:725: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(adata.obs[region_key]):
/tmp/ipykernel_162779/2469090241.py:8: UserWarning: Converting `region_key: region` to categorical dtype.
  mdata = TableModel.parse(mdata, region=['msi', 'rna'], region_key='region', instance_key='instance_id')


In [8]:
sdata.table

MuData object with n_obs × n_vars = 6125 × 33823
  obs:	'region', 'instance_id'
  uns:	'spatialdata_attrs'
  2 modalities
    rna:	3120 x 32285
      obs:	'in_tissue', 'array_row', 'array_col', 'x', 'y', 'lesion', 'region'
      var:	'gene_ids', 'feature_types', 'genome'
      uns:	'spatial'
      obsm:	'spatial'
    msi:	3005 x 1538
      obs:	'leiden', 'array_row', 'array_col', 'x', 'y'
      var:	'mean', 'std'
      uns:	'leiden_colors', 'spatial'
      obsm:	'X_pca', 'spatial'
      varm:	'PCs'
      obsp:	'connectivities', 'distances'

In [10]:
sdata.shapes

{'msi':                          geometry  radius
 0      POINT (14461.525 5784.610)       1
 1      POINT (13883.064 5784.610)       1
 2      POINT (13304.603 5784.610)       1
 3      POINT (16196.908 6363.071)       1
 4      POINT (15618.447 6363.071)       1
 ...                           ...     ...
 3000  POINT (31236.893 41649.191)       1
 3001  POINT (30658.432 41649.191)       1
 3002  POINT (30079.971 41649.191)       1
 3003  POINT (29501.510 41649.191)       1
 3004  POINT (28923.049 41649.191)       1
 
 [3005 rows x 2 columns],
 'rna':                          geometry  radius
 0     POINT (34740.000 30651.000)       1
 1      POINT (7784.000 27046.000)       1
 2     POINT (33275.000 36197.000)       1
 3     POINT (38516.000 28139.000)       1
 4     POINT (41113.000 32686.000)       1
 ...                           ...     ...
 3115  POINT (23757.000 17504.000)       1
 3116   POINT (6074.000 15942.000)       1
 3117  POINT (27517.000 21045.000)       1
 3118  POINT

In [14]:
sd.transformations.get_transformation_between_coordinate_systems(sdata, 'rna', 'msi')

NodeNotFound: source node rna not in graph